# Lottery ticket exploration

This notebook explores lottery tickets and how they could be manipulated.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'dogwood', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('../..')

## Exploration

### Train a model on MNIST

In [ ]:
import numpy as np
import plotly.express as px
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Layer

In [ ]:
MNIST_IMAGE_SHAPE = (28, 28)
MAX_PIXEL_VALUE = 255
MODEL_SAVE_DIR = '/tmp/dogwood/mnist'

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = tf.cast(X_train, tf.float32) / MAX_PIXEL_VALUE
X_test = tf.cast(X_test, tf.float32) / MAX_PIXEL_VALUE
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
model = Sequential([
    Flatten(input_shape=(MNIST_IMAGE_SHAPE), name='flatten'),
    Dense(128, activation='relu', name='dense_1'),
    Dense(10, activation='softmax', name='dense_2')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['sparse_categorical_accuracy'])

In [ ]:
dense_1_initial_weights_and_biases = model.layers[1].get_weights()
dense_1_initial_weights = dense_1_initial_weights_and_biases[0]
dense_2_initial_weights_and_biases = model.layers[2].get_weights()
dense_2_initial_weights = dense_2_initial_weights_and_biases[0]

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
model.evaluate(X_test, y_test)

### Identify lottery tickets

According to Frankle and Carbin (2019), we can identify a winning ticket almost trivially by performing the following steps.

1. Randomly initialize a neural network.
1. Train the network.
1. Prune p% of the parameters in the trained weights, creating a mask. The lowest-magnitude parameters are trimmed.
1. Reset these parameters to their randomly initialized values, creating the winning ticket.

In [ ]:
def get_prune_mask(weights: np.ndarray, prune_rate: float) -> np.ndarray:
    """Returns a mask representing the pruned parameters.
    
    Pruned parameters are given a value of 0 in the mask; retained parameters
    are given a value of 1. The lowest-magnitude parameters are pruned. If
    prune_rate is set to 0.9, then 90% of parameters are masked.
    
    :param weights: The weights of one layer in a neural network.
    :param prune_rate: The fraction of parameters to prune in the range [0, 1].
    :return: A binary mask representing the pruned parameters.
    """
    # TODO I think we need to do abs first.
    min_keep = np.quantile(weights, prune_rate)
    return weights >= min_keep

In [ ]:
dense_1_initial_weights

In [ ]:
np.quantile(dense_1_initial_weights, 0.99)

In [ ]:
dense_1_initial_weights.min()

In [ ]:
dense_1_initial_weights.max()

In [ ]:
(dense_1_initial_weights > np.quantile(dense_1_initial_weights, 0.92)).mean()

In [ ]:
dense_1_mask = get_prune_mask(dense_1_initial_weights, 0.9)
dense_1_mask.mean()

In [ ]:
dense_1_mask.shape

In [ ]:
dense_2_mask = get_prune_mask(dense_2_initial_weights, 0.9)
dense_2_mask.mean()

In [ ]:
dense_2_mask.shape

### Visualize lottery tickets

In [ ]:
dense_1_mask_rect = dense_1_mask.reshape((28, 28, 128))
dense_1_mask_rect.shape

In [ ]:
fig = px.imshow(dense_1_mask_rect.mean(axis=-1))
fig.show()

In [ ]:
fig = px.imshow(dense_2_mask, zmin=0, zmax=1)
fig.show()